In [1]:
!pip show wordcloud

Name: wordcloud
Version: 1.9.2
Summary: A little word cloud generator
Home-page: https://github.com/amueller/word_cloud
Author: Andreas Mueller
Author-email: t3kcit+wordcloud@gmail.com
License: MIT
Location: /home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages
Requires: matplotlib, numpy, pillow
Required-by: stylecloud


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle, ast

from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline, DataCollatorWithPadding
import torch
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

torch.cuda.is_available()

ImportError: cannot import name 'UMAP' from 'umap' (/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/__init__.py)

In [ ]:
# Read the excel review file
df = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_labelled.csv")
df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_sentiment_labelled.csv")
df3 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_emotions_labelled.csv")

# Put the data in the correct format for bertopic
docs = df3["allComment"].astype(str).tolist()

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, max_length=512, truncation=True)

predictions = []

for review in [[x] for x in df.allComment.tolist()]:
    prediction = classifier(review)
    predictions.append(prediction)

res = pd.DataFrame([item for sublist in predictions for item in sublist])
df_pred = pd.concat([df, res], axis=1)

In [ ]:
df2 = df_pred.copy()
# Rename the second 'label' column to 'sentiment_label'
cols = df2.columns.tolist()
cols[len(cols) - 1 - cols[::-1].index('label')] = 'sentiment_label'
df2.columns = cols
df2.columns
# df2.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_sentiment_labelled.csv")

In [ ]:
# Check if a CUDA-enabled GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, problem_type="multi_label_classification", max_length=512)

# Move the model to the GPU
model = model.to(device)

# Set the batch size
batch_size = 2

# Create a list of label names
label_emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise','neutral']

# Initialize lists to store the predicted labels and scores
predicted_labels = []
predicted_scores = []

df3 = df2
# Iterate over the rows of the DataFrame in batches
for i in range(0, len(df3), batch_size):
    batch = df3[i:i+batch_size]
    texts = batch['allComment'].tolist()
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    
    # Move the inputs to the GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    outputs = model(**inputs)
    probs = outputs.logits.sigmoid().detach().cpu().numpy()
    
    # Apply a threshold to the probabilities to get the predicted labels
    threshold = 0.5
    labels = [[label_emotions[i] for i, prob in enumerate(prob_row) if prob > threshold] for prob_row in probs]
    
    # Store the predicted labels and scores
    predicted_labels.extend(labels)
    scores = [{label_emotions[i]: prob for i, prob in enumerate(prob_row)} for prob_row in probs]
    # predicted_scores.extend(probs.tolist())
    predicted_scores.extend(scores)

# Add the predicted labels and scores as new columns in the DataFrame
df3['predicted_labels'] = predicted_labels
df3['predicted_scores'] = predicted_scores

# Convert the string values in the 'predicted_labels' column into lists
df3['predicted_labels'] = df3['predicted_labels'].apply(lambda x: ast.literal_eval(x))
# Convert the string values in the 'predicted_scores' column into dictionaries
df3['predicted_scores'] = df3['predicted_scores'].apply(lambda x: ast.literal_eval(x))

# df3.to_csv("/media/cattiaux/DATA/Wassati/team_data/schneider/df_emotions_labelled.csv")

In [ ]:
df3.single_emotion_label.unique()

In [ ]:
positive_emotions = ['admiration','approval','gratitude','caring','realization','joy','optimism','love','excitement','amusement','relief']
negative_emotions = ['disappointment','disapproval','annoyance','confusion','nervousness','fear','sadness','remorse','disgust','embarrassment','anger']
neutral_emotions = ['neutral','desire','surprise','curiosity']

# Convert the string values in the 'predicted_labels' column into lists
df3['predicted_labels'] = df3['predicted_labels'].apply(lambda x: ast.literal_eval(x))
# Convert the string values in the 'predicted_scores' column into dictionaries
df3['predicted_scores'] = df3['predicted_scores'].apply(lambda x: ast.literal_eval(x))

# Create a new column 'single_emotion_label' that contains the label with the highest score
df3['single_emotion_label'] = df3['predicted_scores'].apply(lambda x: max(x, key=x.get))
# Create a new column 'sentiment' that contains the sentiment of the emotion in the 'single_emotion_label' column
df3['single_sentiment_label'] = df3['single_emotion_label'].apply(lambda x: 'positive' if x in positive_emotions else ('negative' if x in negative_emotions else ('neutral' if x in neutral_emotions else 'unknown')))

# Use the explode() method to transform each element of the list into a row
new_df3 = df3.explode('predicted_labels')
new_df3 = new_df3.rename(columns={'predicted_labels': 'emotion_label'})

# Pour retourner au df initial avec une liste de label
# Group the rows by the original index and aggregate the 'emotion_label' column into a list
# original_df = new_df.groupby(new_df.index).agg({'emotion_label': list})
new_df3.tail()

In [ ]:
# Calculate the total number of rows for each sentiment in the 'sentiment_label' column
total_positive = len(df3[df3['sentiment_label'] == 'positive'])
total_negative = len(df3[df3['sentiment_label'] == 'negative'])
total_neutral = len(df3[df3['sentiment_label'] == 'neutral'])

# Calculate the number of matches for each sentiment
positive_matches = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_label'] == 'positive')])
negative_matches = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_label'] == 'negative')])
neutral_matches = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_label'] == 'neutral')])

# Calculate the percentage of matches for each sentiment
positive_match_percent = positive_matches / total_positive * 100 if total_positive > 0 else 0
negative_match_percent = negative_matches / total_negative * 100 if total_negative > 0 else 0
neutral_match_percent = neutral_matches / total_neutral * 100 if total_neutral > 0 else 0

# Calculate the number of differences for each sentiment
positive_differences = total_positive - positive_matches
negative_differences = total_negative - negative_matches
neutral_differences = total_neutral - neutral_matches

# Calculate the percentage of differences for each sentiment
positive_difference_percent = positive_differences / total_positive * 100 if total_positive > 0 else 0
negative_difference_percent = negative_differences / total_negative * 100 if total_negative > 0 else 0
neutral_difference_percent = neutral_differences / total_neutral * 100 if total_neutral > 0 else 0

# Calculate the distribution of differences for each sentiment
positive_to_negative = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_label'] == 'negative')])
positive_to_neutral = len(df3[(df3['sentiment_label'] == 'positive') & (df3['single_sentiment_label'] == 'neutral')])
negative_to_positive = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_label'] == 'positive')])
negative_to_neutral = len(df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_label'] == 'neutral')])
neutral_to_positive = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_label'] == 'positive')])
neutral_to_negative = len(df3[(df3['sentiment_label'] == 'neutral') & (df3['single_sentiment_label'] == 'negative')])

# Print the results
print(f"Positive matches: {positive_match_percent:.2f}%")
print(f"Negative matches: {negative_match_percent:.2f}%")
print(f"Neutral matches: {neutral_match_percent:.2f}%")
print(f"Positive differences: {positive_difference_percent:.2f}%")
print(f"Negative differences: {negative_difference_percent:.2f}%")
print(f"Neutral differences: {neutral_difference_percent:.2f}%")
print(f"Positive to Negative: {positive_to_negative / total_positive * 100:.2f}%" if total_positive > 0 else "Positive to Negative: N/A")
print(f"Positive to Neutral: {positive_to_neutral / total_positive * 100:.2f}%" if total_positive > 0 else "Positive to Neutral: N/A")
print(f"Negative to Positive: {negative_to_positive / total_negative * 100:.2f}%" if total_negative > 0 else "Negative to Positive: N/A")
print(f"Negative to Neutral: {negative_to_neutral / total_negative * 100:.2f}%" if total_negative > 0 else "Negative to Neutral: N/A")
print(f"Neutral to Positive: {neutral_to_positive / total_neutral * 100:.2f}%" if total_neutral > 0 else "Neutral to Positive: N/A")
print(f"Neutral to Negative: {neutral_to_negative / total_neutral * 100:.2f}%" if total_neutral > 0 else "Neutral to Negative: N/A")

# Filter the DataFrame to only include rows where the 'sentiment_label' is 'negative' and the 'single_sentiment_label' is 'neutral'
negative_to_neutral = df3[(df3['sentiment_label'] == 'negative') & (df3['single_sentiment_label'] == 'neutral')]

# Calculate the distribution of emotions in the 'single_emotion_label' column
emotion_distribution = negative_to_neutral['single_emotion_label'].value_counts(normalize=True) * 100

# Print the results
emotion_distribution

In [ ]:
def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

In [ ]:
import copy

topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

# names = ["Delivery Deadlines",
#     "Pricing", #Quotation and Pricing Strategies
#     "Touch Screens", #Touch Panels and Screens
#     "Frequency Converters",
#     "Automation Components",
#     "Product Evaluation",
#     "Customer Support", #Reliability and Quality in Customer Service and Support
#     "Quick Customer Service",
#     "Problem Solving & Comm",
#     "Assistance", #Assistance and Guidance
#     "Power Supply Issues",
#     "Technical Support",
#     "positive feedback"]

# # Create a dictionary where the keys are the topics and the values are the custom labels
# topic_labels_dict = {}
# topic_labels_dict[-1]="Outliers"
# for i in range(len(topics_to_merge)):
#     for topic in topics_to_merge[i]:
#         topic_labels_dict[topic] = names[i]

topic_model_merged = copy.deepcopy(topic_model)
topic_model_merged.merge_topics(docs, topics_to_merge)
# topic_model_merged.set_topic_labels(topic_labels_dict)

# topic_model_merged.visualize_barchart(top_n_topics=50, custom_labels=True)

In [ ]:
label_names = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]
mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names)}

topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

topic_model_merged.get_topic_info()

In [ ]:
# visualize the topic representation of major topics per class:
topics_per_sentiment = topic_model_merged.topics_per_class(df3["allComment"].astype(str).tolist(), classes=df3.sentiment_label.to_list())
topic_model_merged.visualize_topics_per_class(topics_per_sentiment, custom_labels=True)

In [ ]:
# visualize the topic representation of major topics per class:
topics_per_emotion = topic_model_merged.topics_per_class(df3["allComment"].astype(str).tolist(), classes=df3.single_emotion_label.to_list())
topic_model_merged.visualize_topics_per_class(topics_per_emotion, top_n_topics=30, custom_labels=True)

In [ ]:
# visualize the topic representation of major topics per class:
topics_per_emotion = topic_model_merged.topics_per_class(df3["allComment"].astype(str).tolist(), classes=df3.market_segment.to_list())
topic_model_merged.visualize_topics_per_class(topics_per_emotion, top_n_topics=30, custom_labels=True)

In [ ]:
# Run the visualization with the original embeddings
topic_model_merged.visualize_documents(docs, embeddings=embeddings, custom_labels=True, hide_annotations=True)

In [ ]:
from umap import UMAP

# Reduce dimensionality of embeddings, this step is optional
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model_merged.visualize_documents(docs, reduced_embeddings=reduced_embeddings, custom_labels=True, hide_annotations=True)


In [ ]:
# Convertir la colonne de date en un objet datetime
df3['Response Date'] = pd.to_datetime(df3['Response Date'])
# Extraire l'année et le mois et les stocker dans une nouvelle colonne
df3['year_month'] = df3['Response Date'].dt.to_period('M').dt.to_timestamp()

topics_over_time = topic_model_merged.topics_over_time(docs, df3['year_month'])#, nr_bins=20)
topic_model_merged.visualize_topics_over_time(topics_over_time, custom_labels=True)

In [ ]:
topic_model_merged.visualize_heatmap(custom_labels=True, n_clusters=4)

In [ ]:
topic_model.visualize_heatmap(n_clusters=8)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model_merged.visualize_topics()

In [ ]:
df3.columns